In [1]:
import pandas as pd, numpy as np, json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import bs4
from bs4.element import NavigableString,Tag
pd.set_option('display.width', 1000)

In [2]:
config = pd.read_json('config.json',typ='series')

In [3]:
unis = pd.read_excel('20190912 Web search.xlsx', sheet_name = 0).dropna()

In [4]:
unis

,Institution,Location,# graduates per year
0,Bucknell University,"Lewisburg, PA",900.0
1,Carleton College,"Northfield, MN",500.0
3,Claremont McKenna College,"Claremont, CA",500.0
4,Coe College,"Cedar Rapids, IA",300.0
5,Cornell College,"Mt Vernon, IA",200.0
6,Drake University,"Des Moines, IA",750.0
8,Grinnell College,"Grinnell, IA",400.0
9,Lake Forest College,"Lake Forest, IL",250.0
10,Luther College,"Decorah, IA",500.0
11,Macalester College,"St. Paul, MN",700.0


In [16]:
google_results['email'] = ''

In [22]:
linkedin_urls = []
driver = webdriver.Chrome('chromedriver.exe')
# driver.get('https://www.linkedin.com')

driver.get('https://www.linkedin.com/in/aravind-venugopal/')
sleep(2)
# sign_in_button = driver.find_element_by_class_name('nav__button-secondary')
# sign_in_button.click()
# sleep(2)

sign_in_button = driver.find_element_by_link_text("Sign in")
sign_in_button.click()
sleep(1)

username = driver.find_element_by_id('username')
username.send_keys(config.username)
sleep(2)

password = driver.find_element_by_id('password')
password.send_keys(config.password)
sleep(1)

# username = driver.find_element_by_class_name('login-email')
# username.send_keys(config.username)
# sleep(2)

# password = driver.find_element_by_class_name('login-password')
# password.send_keys(config.password)
# sleep(1)

sign_in_button = driver.find_element_by_xpath('//*[@type="submit"]')
sign_in_button.click()
sleep(7)

emails = []

ind = max(google_results.loc[google_results['email']=='na'].index)
print('startID:',ind)
for student in google_results['Links'][ind:ind+300]:
# for student in ['https://www.linkedin.com/in/aravindvenugopal/']:
    driver.get(student)
    sleep(2)
    try:
        contact_button = driver.find_element_by_link_text("Contact info")
        contact_button.click()
        sleep(1)
    except:
        continue
    html = driver.page_source
    soup = bs4.BeautifulSoup(html)
    for link in soup.find_all("a"):
        url = link.get('href')
        email = 'na'
        if 'mailto' in url:
            email = url.split(':')[1]
            print(url)
            break
    google_results.loc[google_results['Links']==student,'email'] = email
    emails.append(email)
print('Done')
# print('endID:',ind)

startID: 1184


WebDriverException: Message: chrome not reachable
  (Session info: chrome=77.0.3865.90)


In [18]:
max(google_results.loc[google_results['email']=='na'].index)

363

In [25]:
google_results['email'].value_counts()

na                                             1075
ian@iantslagle.me                                 1
mstuppiello21@students.claremontmckenna.edu       1
2684166772@qq.com                                 1
ebech72@gmail.com                                 1
bsubucknell@gmail.com                             1
Name: email, dtype: int64

In [24]:
google_results.head()

,School,Links,email
0,Bucknell University,https://www.linkedin.com/in/tyler-marchiano-b6...,NaN
1,Bucknell University,https://www.linkedin.com/in/craig-silverman-b0...,NaN
2,Bucknell University,https://www.linkedin.com/in/michael-drabich-a8...,NaN
3,Bucknell University,https://www.linkedin.com/in/patricha-williams-...,NaN
4,Bucknell University,https://www.linkedin.com/in/hongyi-wang-3991a3129,NaN


In [6]:
google_results = pd.read_csv('results_part2.csv').iloc[:,1:]

In [20]:
len(google_results)

2998

In [17]:
def search_google_by_school(school):
    linkedin_urls = []
    query = 'site:linkedin.com/in/ AND "student" AND "{}"'.format(school)
    
    driver = webdriver.Chrome('chromedriver.exe')
    driver.get('https:www.google.com')
    sleep(0.5)

    search_query = driver.find_element_by_name('q')
    search_query.send_keys(query)
    sleep(0.5)

    search_query.send_keys(Keys.RETURN)
    sleep(1)

    for page in range(0,20):
        html = driver.page_source
        soup = bs4.BeautifulSoup(html)
        for link in soup.find_all("a"):
            url = link.get('href')
            if url != None:
                if url.startswith('https://www.linkedin.com/'):
                    linkedin_urls.append(url)

        sleep(0.5)

        #For next page in Google results
        next_page = driver.find_element_by_id('pnnext')
        next_page.click()

        sleep(1)

    driver.quit()
    return linkedin_urls

In [16]:
google_results = google_results.reset_index(drop = True)

In [23]:
google_results.to_csv('results_part3.csv')

In [18]:
google_results['School'].unique()

array(['Bucknell University', 'Carleton College',
       'Claremont McKenna College', 'Coe College', 'Cornell College',
       'Drake University', 'Grinnell College', 'Lake Forest College',
       'Luther College', 'Macalester College', 'Pomona College',
       'St. Olaf College', 'Vassar College', 'Wesleyan University',
       'Westmont College'], dtype=object)

In [28]:
google_results = pd.read_csv('results_part2.csv').iloc[:,1:]

In [32]:
google_results['Links'].iloc[1]

'https://www.linkedin.com/in/craig-silverman-b03415126'

In [19]:
# google_results = pd.DataFrame({'School':[],'Links':[]})
for school in unis['Institution'].values.tolist()[:2]:
    if school in google_results['School'].values.tolist():
        links = search_google_by_school(school)
#         google_results = google_results.append(pd.DataFrame({'School':school, 'Links': links}))

WebDriverException: Message: chrome not reachable
  (Session info: chrome=77.0.3865.90)
